# 读取数据

In [2]:
from gspan_mining.config import parser
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from random import random
import sys

import numpy as np
import pandas as pd


import random 
from tqdm import tqdm

In [6]:
import os
os.getcwd()

'/Users/tangting/Project/gSpan-master'

In [8]:
int(0.8*2)

1

# 1. 生成 Vertex 数据集

In [12]:
vertex_count = 1000
origin_ids = []
int_fetaures = []
float_fetaures = []
node_label_0 = []
node_label_1 = []
for i in range(vertex_count):
    origin_id = random.randint(1000000,100000000)
    if origin_id not in origin_ids:
        origin_ids.append(origin_id)
        int_fetaures.append([random.randint(10,100),random.randint(200,300)])
        float_fetaures.append([random.random()*10,random.random()*2000])
        node_label_0.append(int(random.random()*2))    
        node_label_1.append(int(random.random()*2))



# vertext_tb = pd.DataFrame(data=[origin_ids,int_fetaures,float_fetaures],columns=["origin_id","int_fetaures","float_fetaures"])
vertext_tb = pd.DataFrame({"origin_id" : origin_ids, \
                            "int_fetaures": int_fetaures, \
                            "float_fetaures" :float_fetaures,
                            "node_label_0":node_label_0,\
                           "node_label_1":node_label_1     })
vertext_tb["id"] = vertext_tb.index
vertext_tb.to_csv("data/vertex_data.csv",index=False,header=True)


In [15]:
src_rand_id

1000

# 2. 生成 Edge 数据集


In [20]:
edge_count = vertex_count * 10
edge_src_ids = []
edge_dst_ids = []
edge_types = []
for i in range(edge_count):
    src_rand_id = random.randint(0,vertex_count-1)
    dst_rand_id = random.randint(0,vertex_count-1)
    if src_rand_id != dst_rand_id:
        src_id = origin_ids[src_rand_id]
        edge_src_ids.append(src_id)
        dst_id = origin_ids[dst_rand_id]
        edge_dst_ids.append(dst_id)
        edge_types.append(random.randint(0,5))

# vertext_tb = pd.DataFrame(data=[origin_ids,int_fetaures,float_fetaures],columns=["origin_id","int_fetaures","float_fetaures"])
edge_tb = pd.DataFrame({"src_origin_id" : edge_src_ids,"dst_origin_id": edge_dst_ids,"edge_type" :edge_types})

edge_tb.to_csv("data/edge_data.csv",index=False,header=True)


In [25]:

edge_tb_src = edge_tb.merge(vertext_tb[["id","origin_id"]], \
                            left_on="src_origin_id", \
                            right_on="origin_id", \
                            how="left") \
                                [["src_origin_id","dst_origin_id","edge_type","id"]].rename(columns = {"id":"src_id"})

edge_tb_dst = edge_tb_src.merge(vertext_tb[["id","origin_id"]], \
                            left_on="dst_origin_id", \
                            right_on="origin_id", \
                            how="left") \
                                [["src_origin_id","dst_origin_id","edge_type","src_id","id"]].rename(columns = {"id":"dst_id"})

edge_tb_dst

# edge_tb


,src_origin_id,dst_origin_id,edge_type,src_id,dst_id
0,51793311,11371324,2,713,756
1,23112449,70420891,2,701,870
2,20135911,39187452,3,999,482
3,67816312,32079236,4,265,524
4,85452032,13281171,1,943,137
...,...,...,...,...,...
9988,68572401,28792637,0,337,261
9989,70995900,77854936,1,447,536
9990,49589819,78200559,0,769,551
9991,6613341,82716460,5,168,948


# 3.修改edge_id

# 4. 其他处理

In [124]:
# 各项参数
graph_dict =  {"group_id":"graph_id","vertex_id":"id","edge_src":"src","edge_dst":"dst"}
database_file_name = "data/"

In [125]:
graph_id = graph_dict['group_id']
vertex_id = graph_dict['vertex_id']
edge_src = graph_dict['edge_src']
edge_dst = graph_dict['edge_dst']

edges = pd.read_csv(database_file_name+"/edges.csv", sep=',')
vertexes = pd.read_csv(database_file_name+"/vertexes.csv", sep=',')
vertexes.fillna(0)
vertexes.rename(columns={graph_id: 'group_id', vertex_id: "vertex_id"}, inplace=True)

vertexes["node_label_change"] = vertexes["node_label_1"] * 2+vertexes["node_label_0"]

edges.rename(columns={edge_src: 'edge_src',
                edge_dst: "edge_dst"}, inplace=True)

graphs = vertexes["group_id"].drop_duplicates()
graphs_len = len(graphs)
for i in tqdm(range(graphs_len)):
    g_id = graphs.iloc[i]
    vertexes_pd = vertexes[vertexes.group_id == g_id]
    mask = edges[edges.edge_src.isin(vertexes_pd["vertex_id"])
                        & edges.edge_dst.isin(vertexes_pd["vertex_id"])].index
    edges.loc[mask,"group_id"] =  g_id
    edges.loc[mask,"e_id"] =  [i for i in range(len(mask))]

100%|██████████| 41/41 [00:00<00:00, 174.24it/s]


vertexes

In [127]:
# vertexes
# edges
edges_src = pd.merge(edges,vertexes[["group_id","vertex_id","node_label_change"]],how="left",left_on="edge_src",right_on="vertex_id")

# .merge(vertexes,how="left",left_on="edge_dst",right_on="vertex_id") 
edges_src.rename(columns={"group_id":"group_id_a","vertex_id":"vertex_id_a","node_label_change":"node_label_change_a"},inplace=True)
edges_src = edges_src[edges_src.group_id_x==edges_src.group_id_y]

edges_dst = edges_src.merge(vertexes[["group_id","vertex_id","node_label_change"]],how="left",left_on="edge_dst",right_on="vertex_id")
edges_dst.rename(columns={"group_id":"group_id_b","vertex_id":"vertex_id_b","node_label_change":"node_label_change_b"},inplace=True)
edges_dst = edges_dst[edges_dst.group_id_x==edges_dst.group_id_b]

edges_final = edges_dst[["edge_src", 'edge_dst', 'edge_type', 'group_id_x', 'e_id',
        'node_label_change_a',  'node_label_change_b']]

In [137]:

vlb_counter = vertexes.groupby("node_label_change")["vertex_id"]
vlb_counted = vertexes.groupby(["node_label_change","group_id"])["vertex_id"]

In [138]:
vevlb_counter = edges_final.groupby(["node_label_change_a","edge_type","node_label_change_b"])["e_id"]
vevlb_counted = edges_final.groupby(["group_id_x","node_label_change_a","edge_type","node_label_change_b"])["e_id"]

In [146]:
b = vevlb_counter.count().reset_index()
# b

get_forward_root_edges result

In [147]:
get_forward_first = edges_final[edges_final.edge_src<edges_final.edge_dst]


In [148]:
# vevlb:(v.vlb, e.elb, g.vertices[e.to].vlb) , 
# projected:PDFS(gid, e, None)

get_forward_first_start = get_forward_first[get_forward_first.node_label_change_a==1]
get_forward_first_start["frm"] = 0
get_forward_first_start["to"] = 1



,edge_src,edge_dst,edge_type,group_id_x,e_id,node_label_change_a,node_label_change_b
1,22,26,3,20.0,0.0,3,1
3,11,18,1,10.0,0.0,0,1
6,23,30,5,0.0,3.0,1,3
10,9,12,4,0.0,5.0,1,0
13,23,24,2,20.0,2.0,2,3
...,...,...,...,...,...,...,...
84066,4925,4934,0,490.0,2968.0,3,2
84067,498,4946,1,490.0,2969.0,3,2
84070,4912,4934,1,490.0,2972.0,3,2
84071,498,4935,3,490.0,2973.0,3,3


In [62]:
edges.loc[mask,"e_id"] =  [i for i in range(len(mask))]

In [63]:
edges

,Unnamed: 0,edge_src,edge_dst,edge_type,group_id,e_id
0,0,112,134,4,0,NaN
1,1,158,121,3,0,NaN
2,2,150,148,2,0,NaN
3,3,117,13,3,0,NaN
4,4,125,150,2,0,NaN
...,...,...,...,...,...,...
57634,57634,4934,4937,0,64,64.0
57635,57635,4923,498,0,65,65.0
57636,57636,4916,495,1,66,66.0
57637,57637,497,4932,2,67,67.0
